## Exploration of the trade data and trade flow matrix construction.

In [11]:
import polars as pl
import ibis
from ibis import _


In [12]:
ibis.options.interactive = True 

In [13]:
baci_c = ibis.read_csv("datasets/BACI/TPc_V202401.csv")

In [ ]:
# baci_e = pl.read_csv("datasets/BACI/TPe_V202401.csv")

In [17]:
print(baci_c.columns)
print(baci_c.describe())


['year', 'group', 'iso3_tp_o', 'iso3_tp_d', 'pref', 'mfn', 'tariff', 'trade_sq_iy', 'trade_sq_yr', 'flag_extra_neg', 'flag_extra_cty', 'flag_extra_avg']


┏━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━
┃ name           ┃ pos   ┃ type    ┃ count    ┃ nulls   ┃ unique  ┃ mode   ┃ mean         ┃ std          ┃ min     
┡━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━
│ string         │ int16 │ string  │ int64    │ int64   │ int64   │ string │ float64      │ float64      │ float64 
├────────────────┼───────┼─────────┼──────────┼─────────┼─────────┼────────┼──────────────┼──────────────┼─────────
│ trade_sq_yr    │     8 │ float64 │ 10987704 │       0 │ 4686778 │ NULL   │ 8.864341e+07 │ 6.159523e+09 │     0.0 
│ flag_extra_neg │     9 │ int64   │ 10987704 │       0 │       2 │ NULL   │ 1.590026e-01 │ 3.656785e-01 │     0.0 
│ flag_extra_cty │    10 │ int64   │ 10987704 │       0 │       2 │ NULL   │ 3.785297e-01 │ 4.850206e-01 │     0.0 
│ flag_extra_avg │    11 │ int64   │ 10987704 │       0 │       2 │ NULL

┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ year  ┃ group  ┃ iso3_tp_o ┃ iso3_tp_d ┃ pref    ┃ mfn      ┃ tariff   ┃ trade_sq_iy ┃ trade_sq_yr ┃ flag_extra_neg ┃ flag_extra_cty ┃ flag_extra_avg ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ int64 │ string │ string    │ string    │ float64 │ float64  │ float64  │ float64     │ float64     │ int64          │ int64          │ int64          │
├───────┼────────┼───────────┼───────────┼─────────┼──────────┼──────────┼─────────────┼─────────────┼────────────────┼────────────────┼────────────────┤
│  1966 │ _26    │ AFG       │ MAR       │    NULL │ 0.651808 │ 0.651808 │        NULL │         0.0 │              0 │              1 │              1 │
│  1966 │ _26    │ AFG       │ ZWE       │    NULL │ 0.433228 │ 0.433228 │         0.0 │         0.0 │              0 │              1 │              1 │
│  1966 │ _26    │ AFG       │ NIC       │    NULL │ 0.107357 │ 0.107357 │         0.0 │         0.0 │              0 │              1 │              1 │
│  1966 │ _26    │ AFG       │ CHL       │     0.0 │ 0.110000 │ 0.110000 │         0.0 │         0.0 │              0 │              1 │              1 │
│  1966 │ _26    │ AFG       │ SWE       │    NULL │ 0.034834 │ 0.034834 │         0.0 │         0.0 │              0 │              1 │              1 │
└───────┴────────┴───────────┴───────────┴─────────┴──────────┴──────────┴─────────────┴─────────────┴────────────────┴────────────────┴────────────────┘

In [ ]:

baci_c.head()

### Inspecting the number of values per year

In [23]:
baci_c.filter(_.year>2010).group_by(_.year).count().order_by(ibis.desc(_.year))

┏━━━━━━━┳━━━━━━━━━━━━━┓
┃ year  ┃ CountStar() ┃
┡━━━━━━━╇━━━━━━━━━━━━━┩
│ int64 │ int64       │
├───────┼─────────────┤
│  2020 │      238950 │
│  2019 │      238986 │
│  2018 │      239004 │
│  2017 │      239013 │
│  2016 │      239013 │
│  2015 │      239022 │
│  2014 │      239031 │
│  2013 │      239040 │
│  2012 │      239049 │
│  2011 │      239049 │
└───────┴─────────────┘

### Extracting Thailands interactions

In [28]:
baci_tha = baci_c.filter(("THA" == _.iso3_tp_o) | ("THA" == _.iso3_tp_d))

In [31]:
print(baci_tha.head())


┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━
┃ year  ┃ group  ┃ iso3_tp_o ┃ iso3_tp_d ┃ pref    ┃ mfn      ┃ tariff   ┃ trade_sq_iy ┃ trade_sq_yr ┃ flag_extra_n
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━
│ int64 │ string │ string    │ string    │ float64 │ float64  │ float64  │ float64     │ float64     │ int64       
├───────┼────────┼───────────┼───────────┼─────────┼──────────┼──────────┼─────────────┼─────────────┼─────────────
│  1966 │ _26    │ AFG       │ THA       │     0.0 │ 0.463608 │ 0.463608 │        NULL │         0.0 │             
│  1967 │ _26    │ AFG       │ THA       │     0.0 │ 0.463608 │ 0.463608 │        NULL │         0.0 │             
│  1968 │ _26    │ AFG       │ THA       │     0.0 │ 0.463608 │ 0.463608 │         0.0 │         0.0 │             
│  1969 │ _26    │ AFG       │ THA       │     0.0 │ 0.463608 │ 0.463608

In [32]:
print(baci_tha.describe())

┏━━━━━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━
┃ name           ┃ pos   ┃ type    ┃ count  ┃ nulls  ┃ unique ┃ mode   ┃ mean         ┃ std          ┃ min     ┃ p2
┡━━━━━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━
│ string         │ int16 │ string  │ int64  │ int64  │ int64  │ string │ float64      │ float64      │ float64 │ fl
├────────────────┼───────┼─────────┼────────┼────────┼────────┼────────┼──────────────┼──────────────┼─────────┼───
│ trade_sq_yr    │     8 │ float64 │ 146349 │      0 │ 105733 │ NULL   │ 7.065935e+07 │ 9.149485e+08 │     0.0 │   
│ flag_extra_neg │     9 │ int64   │ 146349 │      0 │      2 │ NULL   │ 1.314256e-01 │ 3.378663e-01 │     0.0 │   
│ flag_extra_cty │    10 │ int64   │ 146349 │      0 │      2 │ NULL   │ 2.580544e-01 │ 4.375656e-01 │     0.0 │   
│ flag_extra_avg │    11 │ int64   │ 146349 │      0 │      2 │ NULL   │